# Создание модель машинного обучения для предсказании неудач стартапов

## Описание проекта

Построение бизнеса – непростая задача. Все начинается с идеи, которая может помочь решить проблему, с которой сталкиваются люди. Но даже с самой лучшей и инновационной идеей вы не сможете продвинуться достаточно далеко, если у вас нет достаточной финансовой поддержки. Если стартапу не удастся получить достаточную поддержку, особенно на ранней стадии, ему придется закрыться. В 2022 году отсутствие финансирования привело к неудачам 47% стартапов.
Финансирование — это жизненно важная вещь, в которой нуждается стартап, и оно состоит из множества этапов, называемых раундами финансирования (funding rounds):
- pre-seed funding, вероятно, самый важный раунд, на котором компания пускает корни
- first seed funding, на котором компания должна создать фундамент, на котором она стремится иметь устойчивое будущее.
- series A funding. В течение этого периода привилегированные акции продаются инвесторам, которые желают принимать более активное участие в их развитии.
- series B funding. Это четвертый этап, на котором инвесторы проверяют, продолжает ли компания расти, чтобы они могли получить долгосрочную прибыль.
- series C funding, на этом этапе ожидается, что компания, добившаяся успеха на ранней стадии, будет иметь большой входящий доход.
- series D funding, здесь компания может выбрать переход в серию D или на стадию IPO (первичное публичное размещение акций). Переход к этому этапу вместо этапа IPO не является признаком неудачи.
- Стадия IPO — это когда компания перешла в публичную компанию, на которой компания выпускает новые акции для широкой публики, которая теперь может покупать акции бизнеса.

Каждая компания может перейти в следующий раунд только в том случае, если ей удастся завершить текущий раунд, то есть если они собрали достаточно капитала.

Интересная статистика от: https://explodingtopics.com/blog/startup-failure-stats <q>According to the latest data, up to 90% of startups fail. Across almost all industries, the average failure rate for year one is 10%. However, in years two through five, a staggering 70% of new businesses will fail.</q>
То ест грубо говорят большинство новых стартапов не выживают более пять лет.

Согласно с https://www.investopedia.com/articles/personal-finance/040915/how-many-startups-fail-and-why.asp , причинами сбоев стратапа являются:
- У стартапа закончились деньги.
- Целевой рынок неверен.
- проведено недостаточно исследований.
- Установлено плохое партнерство
- Маркетинг был проведен неправильно
- Предприниматель – новичок в своем деле.

## Цель исследования

Поскольку речь идет о больших деньгах, наша цель — с помощью модели машинного обучения предсказать, закроется ли стартап.

## Ход исследования

- Подгатовка данных: Загрузка и изучение общей информации из предоставлено датасета.
- Предоработка данных: Обработка пропущенных значений, корректировка типа данных, дубликатов и других аномалий.
- Исследовательский анализ данных: Изучение основных параметров объектов, их распределение, присутствие выбросов,  явление и обработка аномали
- Анализ корреляции: Исследование связи между признаками чтобы понимать если нужно или нет устранить несколко признаких
- Построение пайплайн с методом оптимизации чтобы отобрать самую лучшую модель для МО
- Анализы важности признаков чтобы понимать какие признаки важны и какие нет
- Попытка увеличть качество модели при помощи устранения не важных призаков
- Изучение пределов модели и если она соответствует ожиданиям закачика
- Используйте нашу лучшую модель с тестовым набором данных, чтобы предсказать, закроются ли стартапы из датасета



In [ ]:
# Библиотеки

!pip install -U seaborn
!pip install -U scikit-learn
!pip install phik
!pip install shap
!pip install category_encoders
!pip install catboost
!pip3 install pycaret

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import phik
import shap

from google.colab import drive
drive.mount("/content/drive")

from collections import Counter

# загружаем модуль SelectKBest
from sklearn.feature_selection import SelectKBest, f_classif

# загружаем модуль пермутации
from sklearn.inspection import permutation_importance

# Encoders
import category_encoders as ce
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

# загружаем класс pipeline
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedRandomForestClassifier

from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder

# загружаем функцию для работы с метриками
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

# импортируем itertools
from itertools import combinations

from pycaret import classification
# from pycaret.utils import enable_colab
# enable_colab()

shap.initjs()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.13.1
    Uninstalling seaborn-0.13.1:
      Successfully uninstalled seaborn-0.13.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 34.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.3 MB/s eta 0:00:00


# Загрузка данные

## Тренировочние данные

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Startups_project/kaggle_startups_train_28062024.csv')
df_train.head()

In [ ]:
df_train.info()

Датафрейм с тренировочнами датами содержит 52516 строки и 13 столбцов, 1 столбцец с количественными данными:

- funding_total_usd, общая сумма финансирования в USD;

и 12 столбцов с категориальными данными:
- name - Название стартапа
- category_list - Список категорий, к которым относится стартап
- status - Статус стартапа (закрыт или действующий)
- country_code - Код страны
- state_code - Код штата
- region - Регион
- city - Город
- funding_rounds - Количество раундов финансирования
- founded_at - Дата основания
- first_funding_at - Дата первого раунда финансирования
- last_funding_at - Дата последнего раунда финансирования
- closed_at - Дата закрытия стартапа (если применимо)

Отметим что ect 1 пропуск в столбце 'name' и многих пропусков в столбце 'category_list', 'funding_total_usd', 'country_code', 'state_code', 'region', 'city' и 'closed_at'.

Отметим что 'name', 'category_list', 'funding_total_usd', 'status', 'country_code', 'state_code', 'region', 'city', 'funding_rounds' в корректном тип дата.
Однако 'founded_at', 'first_funding_at', 'last_funding_at', 'closed_at' нужно их конвертировать в pandas datetime.

'status' - целевой признак. Наше машиное обученое модели являеться классификационны из-за того что целевой признак - категоряльное. Можно тоже думать что деревяний модел будет хорошо работать потому что у нас болшинство категоряльние признаки.

## Тестовочное данные

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Startups_project/kaggle_startups_test_28062024.csv')
df_test.head()

In [ ]:
df_test.info()

По сравню с тренировочной датафреймом, в датафрейме df_test убрали столбце 'status', 'closed_at' и 'founded_at', и появились новий столбце - 'lifetime' (количествие значение). Датафрейм содержит 13125 строки и мы видим  что у нас пропуски в 6 столбцах : 'category_list', 'funding_total_round', 'country_code', 'state_code', 'region' и 'city'.

## Датафрейм с целевым признаком

In [ ]:
y_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Startups_project/kaggle_startups_sample_submit_28062024.csv')
y_test.head()

In [ ]:
y_test.info()

Это датафрейм - наш целевой признак, у него одно и то же индексы как у df_test и они в том же порядке.


# Преоработка данных

## Преобразование типа данных

In [ ]:
dataframes = [df_train, df_test]
columns = ['founded_at', 'first_funding_at', 'last_funding_at', 'closed_at']

for df in dataframes:
    for col in columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')

In [ ]:
df_train.info()

In [ ]:
df_test.info()

Преобразования выполнены успешно.

## Пропуски в датасетах

In [ ]:
df_train.isna().sum()/len(df_train)

Не хватает один имя которое наверно не будем заменить так как мы убираем этот столбец во вермя моделировании. Пропуски в:
- category_list составляет 4,69% данных,
- funding_total_usd составляет 19,17% данных,
- country_code составляет 10,48% данных,
- state_code составляет 12,88% данных,
- region составляет 12,11% данных,
- city составляет 12,11% данных,
- closed_at составляет 90,64% данных


Мы будем заниматься пропуски в столбце 'category_list', 'funding_total_usd', 'country_code', 'state_code', 'region', 'city' во время pipeline с inputer методом.
Пропуски в столбце 'closed_at' будем заниматься в часть 'feature engineering' чтобы создать столбец 'lifetime'.

Давайте посмотрим на пропуски. Мы можем найти некоторые категории с названием компании. Однако что касается отсутствующих стран, мы не можем на самом деле восполнить это, только если сможем угадать их по городам или регионам, если они существовают.

In [ ]:
df_train.query('country_code.isna()==True and state_code.isna()==False or country_code.isna()==True and city.isna()==False '\
              'or country_code.isna()==True and region.isna()==False')

Мы видим, что когда название страны отсутствует, нет соответствующего города или региона.

In [ ]:
df_train['country_code'].value_counts(normalize=True)[:10]

Вероятно, мы будем использовать стратегию «most frequent» с SimpleImputer для признака «country_code».

In [ ]:
df_train['state_code'].value_counts(normalize=True)[:5]

In [ ]:
df_train['region'].value_counts(normalize=True)[:5]

In [ ]:
df_train['city'].value_counts(normalize=True)[:5]

In [ ]:
df_test.isna().sum()

Если страна отсутствует, невозможно угадать, что это такое по другим признаками «state_code», «region» или «city», поскольку они также отсутствуют, когда отсутствует код страны. Если мы посмотрим на 10 наиболее представленных стран, мы увидим, что США представляют 63,2% данных, вторая Великобритания представляет только 6,2%, третья, Канада, представляет 3,2% и десятая, Нидерланды, представляет менее 1%. . Вероятно, стоит использовать опцию 'most_frequent' с SimpleImputer для «country_code». Однако было бы сложнее использовать эту функцию для «state_code», «region» и «city», поскольку наиболее представленные городы, регионы и штаты представляют соответственно только 6% (San Francisco), 15% (San Francisco Bay Area) и 22% (California). Использование стратегии 'most frequent' для этих признаков можеть приведет к искажению набора данных.

In [ ]:
df_train['category_list'].value_counts(normalize=True)[:15]

Что касается признак категории, нам, вероятно, нужно поработать над этой функцией, прежде чем мы сможем узнать, какую стратегию использовать с SimpleImputer. Например, мы видим категории, которые, возможно, придется сгруппировать, например:
'Hardware + Software' и 'Enterprise Software'

Что касается пропущенных числовых значений в 'fund_total_usd', мы собираемся использовать KNNImputer для замены пропущенных значений.

## Дупликаты

### Явние дупликаты

In [ ]:
def find_obvious_duplicates(dataframe):
    return dataframe.duplicated().sum()

dataframes = [df_train]
dataframes_names = ['df_train']

for index, dataframe in enumerate(dataframes):
    res = find_obvious_duplicates(dataframe)
    print(f'{res} явны дупликаты в датафрейме {dataframes_names[index]}')

Нет явных дупликатов.

### Проверка уникальнте значение в столбце 'name'

In [ ]:
def check_duplicates_id(dataframe):
    return dataframe.duplicated(subset='name').sum()

dataframes = [df_train]
dataframes_names = ['df_train', 'df_test']

for index, dataframe in enumerate(dataframes):
    res = check_duplicates_id(dataframe)
    print(f'{res} дупликаты в столбце \'name\' в датафрейме {dataframes_names[index]}')

Все стартапы точно уникальние.

In [ ]:
set(df_train['name'].to_list()).intersection(df_test['name'].to_list())

Эти два набора данных не связаны ни с одной компанией.

### Проверка присутствия не явных дупликатов в других столбцах

In [ ]:
# Список категориальных столбцов
col_cat = ['category_list', 'status', 'country_code', 'state_code', 'region', 'city']

#  Создаем пустой лист для соханиение резултат
list_unique = []

list_unique_cat = {}

for col in col_cat:
    if col in df_train.columns:
        list_unique_cat[col] = df_train[col].unique().tolist()
list_unique.append(list_unique_cat)
list_unique

Похоже, что большая часть значений уникальна, нет дубликатов, за исключением категорий, где есть некоторые слова, написанные иногда с «s», а иногда без ('Application' и 'Applications').
Мы уже понимаем, что столбцы содержают много разных значений, что заставит нас выбрать правильный кодировщик, чтобы избежать проклятия размерности (dimensionnality curse), и правильную стратегию, чтобы наша модель не переобучалось.

## Feature engineering

Поскольку мы собираемся добавить столбцы в наши фреймы данных df_train и df_test, давайте сделаем их копию, чтобы сохранить исходную нетронутой.

In [ ]:
# Копируем датафрейм df_train
X_train = df_train.copy()

In [ ]:
# Копируем датафрейм df_test
X_test = df_test.copy()

## Создание столбец first_funding_year

Поскольку мы не можем закодировать объект datetime, нам нужно извлечь важную информацию из нашей даты. Год может быть важным. Например, в 2008 году случился экономический кризис, и это могло повлиять на данные.

In [ ]:
X_train['first_funding_year'] = X_train['first_funding_at'].dt.year

In [ ]:
X_test['first_funding_year'] = X_test['first_funding_at'].dt.year

## Создание столбец last_funding_year

In [ ]:
X_train['last_funding_year'] = X_train['last_funding_at'].dt.year

In [ ]:
X_test['last_funding_year'] = X_test['last_funding_at'].dt.year

## Создание столбец first_funding_month

Месяц также может иметь значение. Мы знаем, что люди делают ремонт в доме обычно летом, во время Рождества, людям приходится тратить больше и т. д. Вероятно, это не имеет значения в масштабах компании, но мы все равно можем попробовать, в конце мы всегда сможем удалить это, оснобено, если это усложняло нашу модель.

In [ ]:
X_train['first_funding_month'] = X_train['first_funding_at'].dt.month

In [ ]:
X_test['first_funding_month'] = X_test['first_funding_at'].dt.month

## Создание столбец last_funding_month

In [ ]:
X_train['last_funding_month'] = X_train['last_funding_at'].dt.month

In [ ]:
X_test['last_funding_month'] = X_test['last_funding_at'].dt.month

## Создание столбец 'lifetime'

Так как у нас нет столбец 'closed_at' и 'found_at' в X_test и нет столбца 'lifetime' в X_train. Я решил создать столбце 'lifetime' в X_train. Во время моделирование мы будем убирать столбец 'closed_at' из датасета. Дата загрузки - известный, это 2018-01-01, значить что мы можем рачитывать значение столбца 'lifetime'.



In [ ]:
len(X_train.query('status == "operating"'))/len(X_train) == X_train['closed_at'].isna().sum()/len(X_train)

Пропуски в столбце 'closed_at' - компани которие ещё работают.

Во первых, нам нужно создать новый столбец от столбца 'closed_at' где пропуски будут замененые с значение дата загрузки - '2018-01-01'.

In [ ]:
# Преобразоваем строк '2018-01-01' на datetime
loading_date = pd.to_datetime('2018-01-01')

# Заменяем пропуски из 'closed_at' с датой загрузки
X_train['closed_at'] = X_train['closed_at'].fillna(loading_date)

In [ ]:
# Расчитываем значение столбца 'lifetime'
X_train['lifetime'] = X_train['closed_at']-X_train['founded_at']

In [ ]:
# Пробразоваем значение от день до integer
X_train['lifetime'] = X_train['lifetime'].dt.days

In [ ]:
X_train.head()

## Создание столбец pre_seed_last_round

Поскольку год и месяц будут категориялным признаком, у нас нет представления о том, как долго финансирование было. Конечно, разница между первым и последним финансированием не говорит нам, как долго длился последний раунд и длится ли он еще. Помимо этого процента, мы получаем представление о том, как долго длилось время перед первым раудом, что очень важно для стартапа. Кто-то может утверждать, что чем дольше компания будет более успешной, но это гипотеза.

Проверим, что 'lifetime' никогда не бывает равно нулю.

In [ ]:
len(X_train[X_train['lifetime']==0])

Lifetime никогда не бывает равно 0.

In [ ]:
X_train['pre_seed_last_round'] = 1 - (X_train['last_funding_at']-X_train['first_funding_at']).dt.days/X_train['lifetime']

In [ ]:
X_test['pre_seed_last_round'] = 1 - (X_test['last_funding_at']-X_test['first_funding_at']).dt.days/X_train['lifetime']

## Создание столбец countries

Одной из проблем с таким большим количеством отдельных значений будет переобучение: попытка модели соответствовать шуму, категориям с низкой частотой данных. Кроме того, есть еще одна проблема: новые значения из теста не попали в обучающую выборку. Вот почему я собираюсь сгруппировать низкочастотные значения вместе для набора обучающих и тестовых данных, для столбцов 'country_zone' и для других категориальных признаков ('state_code', 'region', 'city') в следующих частях.

Сгруппируем страны с частотой менее 20 вместе в группу 'Misc' ('Miscellaneous').

In [ ]:
n_threshold = 20

In [ ]:
# Подсчет количества одиночных значений в столбце категорий
countries_counts = X_train['country_code'].value_counts()
# Получите индексы объекта из категорий объектов, который отображается меньше порогового числа
countries_count_index = X_train[X_train['country_code'].map(countries_counts)<=n_threshold].index.to_list()
# Нам нужно скопировать 'country_code' в столбец 'countries'
X_train['countries'] = X_train['country_code']
# Замена значения значений, которое меньше порогового числа, на значение 'Misc.'
X_train.loc[countries_count_index, 'countries'] = 'Misc.'

In [ ]:
# Подсчет количества одиночных значений в столбце категорий
countries_counts_test = X_test['country_code'].value_counts()
# Получите индексы объекта из категорий объектов, который отображается меньше порогового числа
countries_count_index_test = X_test[X_test['country_code'].map(countries_counts_test)<=n_threshold].index.to_list()
# Нам нужно скопировать 'country_code' в столбец 'countries'
X_test['countries'] = X_test['country_code']
# Замена значения значений, которое меньше порогового числа, на значение 'Misc.'
X_test.loc[countries_count_index_test, 'countries'] = 'Misc.'

Мы собираемся сделать то же самое для 'state_code', 'region', 'city'.

## Создание столбец states

In [ ]:
states_counts = X_train['state_code'].value_counts()

states_count_index = X_train[X_train['state_code'].map(states_counts)<=n_threshold].index.to_list()

X_train['states'] = X_train['state_code']

X_train.loc[states_count_index, 'states'] = 'misc.'

In [ ]:
states_counts_test = X_test['state_code'].value_counts()

states_count_index_test = X_test[X_test['state_code'].map(states_counts_test)<=n_threshold].index.to_list()

X_test['states'] = X_test['state_code']

X_test.loc[states_count_index_test, 'states'] = 'misc.'

## Создание столбец regions

In [ ]:
regions_counts = X_train['region'].value_counts()

regions_count_index = X_train[X_train['region'].map(regions_counts)<=n_threshold].index.to_list()

X_train['regions'] = X_train['region']

X_train.loc[regions_count_index, 'regions'] = 'misc.'

In [ ]:
regions_counts_test = X_test['region'].value_counts()

regions_count_index_test = X_test[X_test['region'].map(regions_counts_test)<=n_threshold].index.to_list()

X_test['regions'] = X_test['region']

X_test.loc[regions_count_index_test, 'regions'] = 'misc.'

## Создание столбец cities

In [ ]:
cities_counts = X_train['city'].value_counts()#(dropna=False)

cities_count_index = X_train[X_train['city'].map(cities_counts)<=n_threshold].index.to_list()

X_train['cities'] = X_train['city']

X_train.loc[cities_count_index, 'cities'] = 'misc.'

In [ ]:
cities_counts_test = X_test['city'].value_counts()

cities_count_index_test = X_test[X_test['city'].map(cities_counts_test)<=n_threshold].index.to_list()

X_test['cities'] = X_test['city']

X_test.loc[cities_count_index_test, 'cities'] = 'misc.'

## Создание столбец sub_categories

Одна из основных проблем с нашим датафреймом заключается в том, что у нас слишком много уникальных значений, и это может стать проблемой во время кодирования обучение модели. Попробуем уменьшить количество уникальных значений для столбца 'category_list'.

In [ ]:
len(X_train['category_list'].value_counts())

22105 разние категории, это очень много. Давайте групируем несколкие стартары по главнам категорями с помощю словаром.

In [ ]:
cat_dict={
'Biotechnology':['Biotechnology', 'Medical Devices', 'Health Diagnostics', 'Life Sciences', 'Diagnostics'],
'Hardware, Software & Apps': ['Software', 'Enterprise Software', 'Apps', 'Open Source', \
                                'Website and Application Development', 'Web Development', \
                                'Hardware + Software', 'Productivity Software', 'Android', \
                                'Developer APIs', 'Algorithms', 'Hardware', 'iPhone', 'Developer Tools',\
                                'Communications Hardware', 'Browser Extensions', 'iPad', 'iOS',\
                                'Real Time', 'Gamification', 'Application', 'Applications', 'Web'],
'Internet, E-Commerce & Mobile':['Bitcoin', 'E-Commerce', 'Content', 'M-', 'Social Media', \
                                  'Curated Web', 'Internet', 'Online', 'Messaging', 'Search', \
                                  'Marketplaces', 'Wireless', 'Networks', 'Bluetooth', 'Networking', \
                                  'Telecommunications', 'VoIP', 'Development Platforms', \
                                  'Internet of Things', 'Market Research', 'Online Shopping', \
                                  'Online Travel', 'Payments', 'Mobile Payments', 'Mobile', \
                                  'Advertising', 'Consumer Internet', 'Email', 'Social Commerce', \
                                  'Chat', 'All Students', 'Coupons', 'Content Delivery', \
                                  'App Marketing', 'Internet Marketing', 'Social Network Media', 'Gps', \
                                 'Online Dating', 'Service Providers', 'E-Commerce Platforms', \
                                  'Content Discovery', 'Ediscovery', 'App Stores', 'Advertising Networks', \
                                  'Email Marketing', 'Comparison Shopping', 'Online Reservations',\
                                 'Maps', 'Mobile Advertising', 'Online Scheduling', 'Mobile', \
                                  'Bridging Online and Offline', 'Digital Media'],
'Energy':['Energy', 'Oil & Gas', 'Oil', 'Gas'],
'Clean Technology':['Clean Technology', 'Green', 'Environmental Innovation', 'Renewable Energies', 'Solar',
                   'Clean Energy'],
'Health care':['Health Care', 'Healthcare', 'Health', 'Care', 'Health Wellness', 'Medical', 'Hospitals', \
               'Fitness', 'Cannabis', 'Sports', 'Clinical Trials', 'Dental', 'Doctors', 'Exercise',\
               'Medicine', 'Doctor'],
'Entertainment':['Games', 'Game', 'Events', 'Video Games', 'Art', 'Virtual Worlds', 'Video Streaming ', \
                 'Music', 'Augmented Reality', 'Film', 'Concerts', 'Leisure', 'Artists Globally', \
                 'Creative', 'Entertainment Industry', 'Fantasy Sports', 'Angels', 'Celebrity', 'Event', \
                'Online Gaming', 'Comics', 'Entertainement', 'Entertainment', 'Video', 'Audio', \
                 'Nightlife', 'Weddings'],
'Education':['EdTech', 'Education', 'Colleges', 'Universities', 'Educational Games', 'College Campuses',\
            'School'],
'Business & Professional Services':['Finance', 'Consulting', 'Advertising', 'Public Relations', \
                                    'Venture Capital', 'Legal','Local Businesses', 'Startups', \
                                    'Business Services', 'Alumni', 'Enterprises', 'Recruiting', \
                                     'Banking', 'Investment Management', 'Crowdfunding', 'Trading', \
                                    'Outsourcing', 'Small and Medium Businesses', 'Staffing Firms', 'B2B',\
                                    'Business Productivity', 'Advice', 'Logistics', 'Credits', 'Discount',\
                                    'Document Management', 'Reviews and Recommendations', 'Crowdsourcing',\
                                    'Sales and Marketing', 'Freelancers', 'Employer Benefits Programs', \
                                    'Risk Management', 'Business', 'Entrepreneur', 'Collaboration',\
                                    'Accounting', 'Designers', 'Surveys', 'Incubators', 'Distributors',\
                                   'Ad Targeting', 'Billing', 'Employment', 'Contact Management', 'CRM', \
                                    'Marketing', 'Human Resources', 'Career', 'Law',\
                                    'Financial technology', 'FinTech'],
'Governement & Public Services':['Defense', 'Public Transportation', 'Governement', 'Governments',\
                                 'Aerospace', 'Geosptatial', 'Politics'],
'Data Storage & Management':['Big Data', 'Cloud Computing and Storage', 'Data Analytics', 'Data Centers', \
                             'Web Hosting', 'Data Security', 'SaaS', 'Storage', 'Cloud Computing',\
                             'Cloud Data Services', 'Cyber Security', 'Network Security', \
                             'Big Data Analytics', 'Classifiels', 'File Sharing', 'Data Visualization',\
                             'Cloud Management', 'Data Mining', 'Machine Learning', 'Photo Sharing', 'P2P',\
                             'Databases', 'Data', 'Analytics','Security', 'Peer-to-Peer', 'Cloud Security'],
'Consumers & Goods services':['Clothing', 'Fashion', 'Financial Services', 'Photography', \
                                       'Services', 'Consumer Electronics', 'Consumer Goods', 'Consumers',\
                                       'Beauty', 'Home Automation', 'Cars', 'Cosmetics', 'Construction', \
                                       'Location Based Services', 'Utilities', 'Real Estate', 'Home',\
                                       'Collectibles', 'Rental Housing', 'Home Decor', 'Printing', \
                                       'Collaborative Consumption','Auctions', 'Rental',\
                                       'Distribution', 'Commercial Real Estate', 'Transportation', \
                                       'Delivery', 'Pets', 'Babies', 'Carreer Planning', \
                                       'Property Management', 'Electrical Distribution', \
                                       'Lifestyle', 'Consumer Lending', 'Coffee', 'Cooking', 'Goods',\
                                       'Local', 'Health and Insurance', 'Wearable',\
                                       'Bicycles', 'Insurance', 'Kids', 'Domains', 'Baby Accessories',\
                                       'Customer Service', 'Retail', 'Shopping', 'Toys', 'Jewelry', \
                                       'Realtors', 'Lingerie', 'Furniture', 'Spas', 'Women', 'Parking',\
                                      'Service'],
'Chemicals and Materials':['Advanced Materials', 'Chemicals', 'Minerals', 'Pesticides', 'Composites', \
                           'Textiles', 'Materials'],
'Hospitality':['Hospitality', 'Restaurants', 'Reservation', 'Hostels', 'Hotels', 'Travel & Tourism', \
               'Travel', 'Adventure Travel','Tourism'],
'Technology and Industry':['Technologies', 'Technology', 'Tech', 'TECH', 'Nanotechnology', 'Design',
                           'Information Technology', 'Electronics', 'Computers', 'Semiconductors', \
                           '3D Printing', 'Automotive', 'Robotics', \
                           'Communications Infrastructure', 'Industrial Automation', '3D Technology', '3D', \
                           'Architecture', 'Manufacturing', 'Engineering', 'Biometrics', 'Drones', 'Auto',\
                           'Batteries', 'Innovation Engineering', 'Assisitive Technology', \
                           'Engineering Firms', 'Digital Signage', 'Bioinformatics', 'Mining Technologies',\
                           'Boating Industry', 'Infrastructure', 'Industrial', 'IT', 'Vision', \
                           'Artificial Intelligence', 'Vehicles', 'Engineers'],
'Media':['Media', 'News', 'Broadcasting', 'Television', 'Publishing',  'Journalism'],
'Pharma':['Pharmaceuticals', 'Bio-Pharm', 'Therapeutics', 'Drugs'],
'Social':['Nonprofits', 'Non profits', 'Non Profit', 'Communities', 'Charity'],
'Food':['Food Processing', 'Specialty Foods', 'Wine And Spirits', 'Agriculture', 'Farming', \
        'Craft Beer', 'Brewing', 'Organic', 'Fruit'],
'Misc.':['Misc.']
}

In [ ]:
# Инвертируйем ключ и значения, чтобы создать новый словарь для сопоставления значений, извлеченных из столбца списка категорий.
from collections import defaultdict
inv_cat_list = defaultdict(str)

for keys, vals in cat_dict.items():
    for val in vals:
        inv_cat_list[val]=keys

In [ ]:
# Создаем список ключей inv_cat_list
cat_keys = list(inv_cat_list.keys())

In [ ]:
# Извлеките основные категории, появляющиеся в cat_keys
def extract_main_cat(val):

    if val in cat_keys: # Проверим, есть ли значение val в списке cat_keys.
        return val
    elif '|' in val: # категория с '|' внутри
        if val.split('|')[0] in cat_keys:
            return val.split('|')[0]
        elif ' ' in val.split('|')[0]:
            if val.split('|')[0].split(' ')[0] in cat_keys:
                return val.split('|')[0].split(' ')[0]
            elif val.split('|')[0][-1] in cat_keys:
                return val.split('|')[0].split(' ')[-1]
            else:
                if val.split('|')[1] in cat_keys:
                    return val.split('|')[1]
                elif ' ' in val.split('|')[1]:
                    if val.split('|')[1].split(' ')[0] in cat_keys:
                        return val.split('|')[1].split(' ')[0]
                    elif val.split('|')[0].split(' ')[-1] in cat_keys:
                        return val.split('|')[0].split(' ')[-1]
                    else:   # Мы знаем, что всегда второе слово часто принадлежит cat_keys
                        return 'Misc.'
    elif ' ' in val: # категория где именни разделение с пробелом ' '
        if val.split(' ')[0] in cat_keys:
            return val.split(' ')[0]
        else:
            if val.split(' ')[1] in cat_keys:
                return val.split(' ')[1]
            else:
                return 'Misc.'
    elif '&' in val: # категория где именни разделение с '&'
        if val.split('&')[0] in cat_keys:
            return val.split('&')[0]
        else:
            if val.split('&')[-1] in cat_keys:
                return val.split('&')[-1]
            else:
                return 'Misc.'
    elif 'and' in val:   # категория где именни разделение с 'and'
        if val.split('and')[0] in cat_keys:
            return val.split('and')[0]
        else:
            if val.split('and')[-1] in cat_keys:
                return val.split('and')[-1]
            else:
                return 'Misc.'
    else:
        return 'Misc.'


X_train['main_category'] = X_train.query('~category_list.isna()')['category_list'].apply(extract_main_cat)
X_test['main_category'] = X_test.query('~category_list.isna()')['category_list'].apply(extract_main_cat)

In [ ]:
# Создание подкатегорий нашего столбца
X_train['subcategories'] = X_train['main_category'].map(inv_cat_list, na_action='ignore')

In [ ]:
X_train['subcategories'].value_counts()

In [ ]:
X_train[X_train['subcategories']=='Misc.']

In [ ]:
X_test['subcategories'] = X_test['main_category'].map(inv_cat_list, na_action='ignore')

In [ ]:
X_test['subcategories'].value_counts()

## Удаление временного столбца

In [ ]:
X_train.info()

In [ ]:
X_train.drop(['main_category'], axis=1, inplace=True)

In [ ]:
X_test.drop(['main_category'], axis=1, inplace=True)

## Вывод

Мы получили менее 5 % пропущенных значений в «category_list» и около 10 % в «country_code», «state_code», «region», «city». Эти пропущенные значения будут обработаны во время работы паплайны с помощью метода SimpleImputer.

Мы также заметили значительное количество уникальных категориальных значений. Нам пришлось сгруппировать категории стартапов в несколько категорий, которые наиболее информативны. Тогда у нас была очень разная частота, и группировка иногда может вызывать вопросы: должны ли мы отделять категорию "Apps" от категории "Software, Hardware, and Apps"? Должны ли мы объединить категории "Mobile" с категорией "Apps" и почему бы не объединили категорию "Hospitality" с категорией "Customers Goods And Services" ? Однако я не думаю, что это будет иметь большое разницы на MO. В списке категорий мы также группировали категории, которые мы не могли поместить ни в одну группу, в категорию под названием "Misc.".

Мы использовали тот же принцип для признако географического местоположения (country_zone, city и т. д.), те значеня которы редко появились, групировали в группу "Misc.". Мы будем исползовать эту групу во время исползованнии  SimpleImputer, с опцим "constant", потому что я не уверен, что стратегия "most frequent" всегда является хорошей, поскольку она может исказить данные. Например за исключением "США", у нас не так много доминирующих категорий в 'state_code', 'region', 'city'.

Тоже создали дополнителный признак 'pre_seed_last_round'. С начало я хотел создать пизнак которы покажет время перед первым фундом потому что подкотовка стартапы является очень важно что бы предотвратить его неудачу. К сожалению нет признака 'found_at' в тестовом датасете, вот почему создали признак 'pre_seed_last_round', доля которая отрожаеть сколко временни занимались все раундов кроме последние, a другая доля время перед первом раундом + время последнего раунда.


# Исследовательский анализ данных

## Статистика датафреймы

In [ ]:
stat_train = X_train.drop(['first_funding_year','last_funding_year', 'first_funding_month',\
                           'last_funding_month'], axis=1).select_dtypes(include='number').describe()
stat_train

In [ ]:
stat_test = X_test.drop(['first_funding_year','last_funding_year', 'first_funding_month', \
                         'last_funding_month'], axis=1).select_dtypes(include='number').describe()
stat_test

Что касается общего объема финансирования, средние значения между трениворочном и тестовом датасетом аналогичны, однако стандартное отклонение почти в два раза болшее у тренировочной датасета.

Среднее 'lifetime' между датасетами близко, и оно более значительно в треновочном датасете.

Что касается раунда финансирования, то в среднем он составляет от 1 до 2 раундов, и это понятно, поскольку первый и второй раунды являются наиболее важными критическими и сложним проходить для стартапов. Большая часть сбоев стартапов происходит примерно в этом временни. Это и понятно, потому что на этом этапе стартап начнет понимать, сработает ли его бизнес-план, идеи, ответит ли рынок положительно или нет. Наборы тестовых и обучающих данных в этих столбцах близки.

В среднем срок жизни стартапа составляет около 3097 дней, но мы видим, что у нас есть исключения, возможно, успешные компании, которые живут более 17000 дней.

### Выбросы в датасетах

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 7.5))

col_num = ['funding_total_usd', 'funding_rounds', 'lifetime']

axes = axes.flatten()

dataframes = [X_train, X_test]

columns_names = ['funding_total_usd', 'lifetime', ]

sample_type = ['в тренировчне выборке', 'в тестовое выборке']

for i, column in enumerate(X_train[col_num].columns):
    for j, dataframe in enumerate(dataframes):

        sns.boxplot(data=dataframe, x=column, ax=axes[2*i+j])

        axes[2*i+j].set_title(f'Ящик с усами {sample_type[j%2]}')

#plt.title(f'Боксплоты');
plt.tight_layout()


plt.show()


## Проверка дисбаланс для выбросов


Можно подумать, что в выбросах могло бы быть гораздо меньше объектов класса 0 («закрытых»). Конечно, если компания просуществует долго или заработает много денег, это означает, что бизнес прибыльен и вряд ли закроется. В этом случае выбросы могут быть очень полезны для разделения обоих классов. Что касается дисбаланса, можем ли мы вообще подумать об использовании anomaly fraud detection алгоритм для нашей модели, если выбросы помогают дифференцировать оба класса.

In [ ]:
y_test.columns

In [ ]:
columns = ['funding_total_usd', 'lifetime', 'funding_rounds']
s = [('заработовали большее', 'долларов'), ('работали большее', 'дней'), ('проходили больщее','раунды')]

# Индекс для закрыты стартапы в тестовом датасете
index_closed_test = y_test.query('status=="closed"').index.to_list()

for i,col in enumerate(columns):
  for j in range(2):
    if j==0:
      IQR_plus = stat_train.loc['25%',col]+(stat_train.loc['75%',col]\
                                                   -stat_train.loc['25%',col])*1.5
      res = len(X_train.query('{0} > @IQR_plus and status == "closed"'.format(col)))\
      /len(X_train.query('{0} > @IQR_plus'.format(col)))
      print(f'Доля стартапов которы {s[i][0]} {round(IQR_plus,0)} {s[i][1]} и '\
            f'закрыты для тренировочны датасета :{round(res*100,2)}%.')
    else:
      IQR_plus = stat_test.loc['25%',col]+(stat_test.loc['75%',col]\
                                                   -stat_test.loc['25%',col])*1.5
      res = len(X_test.loc[index_closed_test,:].query('{0} > @IQR_plus'.format(col)))\
      /len(X_test.query('{0} > @IQR_plus'.format(col)))
      print(f'Доля стартапов которы {s[i][0]} {round(IQR_plus,0)} {s[i][1]} и '\
            f'закрыты для тестовой датасета :{round(res*100,2)}%.')




Например, можно подумать, что если стартап уже долго существовали, прошла много раундов или заработала много денег, у неё меньше шанс закрывается. Меньшее доли для выбросов выгладить  правильно для тренировочной выборки, однако для тестового набора данных сохраняется баланс 50%-50%, что выглядит подозрительно. Это как будто тестовое датасет содержаеть особые случаи и не отражает реальность.


# Расспределение целевого признака

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

axes = axes.flatten()

dataframes = [X_train['status'].value_counts(normalize=True), y_test['status'].value_counts(normalize=True)]

sample_type = [' в тренировчне выборке', 'в тестовое выборке']

for i, dataframe in enumerate(dataframes):

    sns.barplot(x=dataframe.index, y=dataframe, hue=dataframe.index, ax=axes[i])

    axes[i].set_title(f'Распределение доля {sample_type[i]}')
    #axes[i].set_xticklabels(['Работает', 'Закрыт']);
    axes[i].set_xlabel('Стартап работает ли ?');
    axes[i].set_ylabel('Доли');
    axes[i].grid();


plt.tight_layout()


plt.show()

Мы замечаем огромный дисбаланс в вашем наборе обучающих данных (около 90%–10%), как мы и предвидели в части исследования недостающих данных. Напротив, он хорошо сбалансирован в наборе тестовых данных.

## Исследованние распеделения количеств значений

## Распеделенние общего сбора средств

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

axes = axes.flatten()

dataframes = [X_train, X_test]

sample_type = ['в тренировчне выборке', 'в тестовое выборке']

for i, dataframe in enumerate(dataframes):

    if i == 0:
        sns.histplot(x=dataframe.query('funding_total_usd < 5.000000e+06')['funding_total_usd'], color = 'skyblue',hue=dataframe['status'], \
                     hue_order=['operating', 'closed'], stat='density', common_norm=False, ax=axes[i])
    else:
        sns.histplot(x=dataframe.query('funding_total_usd < 5.000000e+06')['funding_total_usd'], color = 'skyblue',hue=y_test['status'], \
                     hue_order=['operating', 'closed'], stat='density', common_norm=False, ax=axes[i])

    axes[i].set_title(f'Распределение общего сбора средстов {sample_type[i]}')

    axes[i].set_xlabel('Общий сбор средств');
    axes[i].set_ylabel('Плотность распределения');
    axes[i].grid();


plt.tight_layout()


plt.show()

Нам пришлось исползовать заглушку, чтобы остронит выбросов. Мы видим, что график для теста и тренировочной датасета выглядит одинаково. Распределение как распределение Пуассона. Можно сказать что этот признак почти не влияет в цедевое признак для тестовой датасета. Ну видно в первом графике что болшее стартапа закрылись когда мало денег зарабатывали.

## Распеделенние продолжительность жизни

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

axes = axes.flatten()

dataframes = [X_train, X_test]

sample_type = ['в тренировчне выборке', 'в тестовое выборке']

for i, dataframe in enumerate(dataframes):

    if i == 0:
        sns.histplot(x=dataframe['lifetime'], color = 'skyblue',hue=dataframe['status'], \
                     hue_order=['operating', 'closed'], stat='density', common_norm=False, ax=axes[i])
    else:
        sns.histplot(x=dataframe['lifetime'], color = 'skyblue',hue=y_test['status'], \
                     hue_order=['operating', 'closed'], stat='density', common_norm=False, ax=axes[i])

    axes[i].set_title(f'Распределение продолжительность жизни {sample_type[i]}')

    axes[i].set_xlabel('Продолжительность жизни');
    axes[i].set_ylabel('Плотность распределения');
plt.grid();


plt.tight_layout()


plt.show()

Здесь графики совсем не похожы, и мы можем увидеть влияние продолжительности жизни на целевой признак во первом датасете.

Напротив, мы не можем провести различие между двумя классами, и понимать кто живут дольше во втором графике. Мы видим, что в тестовом наборе данных признак "lifetime" почти не влияет на целевую переменную.

Мы говорили что болинство стартапов не живуть болшее 5 лет, и мы видим в обойх графике что это может правда так как у болшиньсто стартапов продолжительность жизни меншее 1825 дней.

Ну сдезсь сложно осудить так как многих стартапов у которых есть меншее 5 летов. Можеть бы много из них будеть закрываеться скоро.


Но есть и кое-что еще, есть разница между датой основания и датой первого финансирования. Мы это не можем проверит в тестовом датасете потому не даты основании ну давайте проверяем с тренировочноы выборком.

In [ ]:
X_train['diff'] = (X_train['first_funding_at']-X_train['founded_at']).dt.days

In [ ]:
fig, axes = plt.subplots(figsize=(15, 5))


sns.histplot(x=X_train.query('0 < diff < 2500')['diff'],hue=X_train['status'], \
hue_order=['operating', 'closed'], stat='density', common_norm=False, ax=axes)

axes.set_title(f'Распределение разница между датой основаной и даты первого финансировании')

axes.set_xlabel('Разница между датой основаной и даты первого финансировании');
axes.set_ylabel('Плотность распределения');
axes.grid();

plt.show()

Мы ясно видим, что у большинства стартапов первый раунд финансирования состоялся менее чем через 750 дней после их основания, то есть примерно через два года. Мы ясно видим, что здесь большая часть стартапов закончилась провалом. Однако по прошествии двух лет, меньше компаний начинают свое финансирование, но и среди них меньше неудачных стартапов, возможно, потому, что им потребовалось больше времени на подготовку. Гипотеза: Увеличивает ли более длительное время предварительной подготовки шансы стартапа на успех?

## Распеделенние доли перед раудом и последннии рауды

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

axes = axes.flatten()

dataframes = [X_train, X_test]

sample_type = ['в тренировчне выборке', 'в тестовое выборке']

for i, dataframe in enumerate(dataframes):

    if i == 0:
        sns.histplot(x=dataframe.query('pre_seed_last_round > 0 and pre_seed_last_round < 1')['pre_seed_last_round'], color = 'skyblue',hue=dataframe['status'], \
                     hue_order=['operating', 'closed'], stat='density', common_norm=False, ax=axes[i])
    else:
        sns.histplot(x=dataframe.query('pre_seed_last_round > 0 and pre_seed_last_round < 1')['pre_seed_last_round'], color = 'skyblue',hue=y_test['status'], \
                     hue_order=['operating', 'closed'], stat='density', common_norm=False, ax=axes[i])

    axes[i].set_title(f'Распределение доли временни перед первом рауды и после плоседнного {sample_type[i]}')

    axes[i].set_xlabel('Доли временние перед первом рауды и после последнного');
    axes[i].set_ylabel('Плотность распределения');
    axes[i].grid();


plt.tight_layout()


plt.show()

На первом графике мы видим компание, у которие было меньше времени на подготовку, и к последнему раунду склонялись к закрытию. На втором графике, опять же, поведение двух классов очень близко, поэтому трудно сказать как признак 'pre_seed_last_round' являет на целевую перемеррую.

Мы заметили некоторые аномалии, когда дата первого раунда наступала после даты основания. Это звучит невозможно. Мы можем думать их убирать.

In [ ]:
X_train.query('first_funding_at < founded_at')

# Исследованние распеделения категориального значения

## Распеделенние количество раунды финансирования

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

axes = axes.flatten()

dataframes = [X_train, X_test]

sample_type = [' в тренировчне выборке', 'в тестовое выборке']

for i, dataframe in enumerate(dataframes):

    if i == 0:
         sns.countplot(x=dataframe['funding_rounds'], hue=dataframe['status'], hue_order=['operating', 'closed'], stat='percent', ax=axes[i]);
    else:

        sns.countplot(x=dataframe['funding_rounds'], hue=y_test['status'], hue_order=['operating', 'closed'], stat='percent', ax=axes[i]);

    axes[i].set_title(f'Распределение количество раунды финансирования {sample_type[i]}')

    axes[i].set_xlabel('Раунды финансирования');
    axes[i].set_ylabel('Количество');
    axes[i].grid();


plt.tight_layout()


plt.show()

Очевидно, что в первом раунде будет больше компаний, потому что это первый и самый простой раунд для входа, но, вероятно, один из самых трудных для выхода, поскольку репутация компании еще не создана. Конечно, по мере увеличения количества раундов стартапов становится все меньше и меньше. К сожалению, мы почти не видим влияния количества раундов на классы целевой переменной в втором графике.



## Распеделенние частота категорий

In [ ]:
fi, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 7.5))

axes = axes.flatten()

dataframes = [X_train, X_test]

sample_type = [' в тренировчне выборке', 'в тестовое выборке']


for i, dataframe in enumerate(dataframes):
    if i == 0:
        list_cat = dataframe['subcategories'].value_counts().index.to_list()[:50]
        index_cat = dataframe.query('subcategories in @list_cat').index.to_list()


        sns.countplot(data=dataframe, x=dataframe.loc[index_cat,'subcategories'], \
                      order=dataframe.loc[index_cat,'subcategories'].value_counts()[:50].index, hue=dataframe['status'], \
                      hue_order=['operating', 'closed'], stat='percent', ax=axes[i]);
    else:
        list_cat = dataframe['subcategories'].value_counts().index.to_list()[:50]
        index_cat = dataframe.query('subcategories in @list_cat').index.to_list()


        sns.countplot(data=dataframe, x=dataframe.loc[index_cat,'subcategories'], \
                      order=dataframe.loc[index_cat, 'subcategories'].value_counts()[:50].index, hue=y_test['status'], \
                      hue_order=['operating', 'closed'], stat='percent', ax=axes[i]);

    axes[i].set_title(f'Распределение частота категорей {sample_type[i]}')
    axes[i].tick_params(axis='x', labelrotation=90)
    axes[i].set_xlabel('Категорий');
    axes[i].set_ylabel('Частота');
    axes[i].grid();


plt.tight_layout()


plt.show()

Стартапы больше всего работают в сфере Internet & E-commerce & Mobileв, затем Hardware, Software and Apps, а затем Business and professional Services. Мы ясно видим дисбаланс наших данных в тренировочном датасета. Во втором графике, разница между работающейвся компанией и закрытой компанией очень мала.

## Распеделенние местоположение

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(15, 15))

axes = axes.flatten()

dataframes = [X_train, X_test]

sample_type = [' в тренировчне выборке', 'в тестовое выборке']
columns = ['countries', 'states', 'regions', 'cities']
column_names = ['страны', 'штаты', 'регионы', 'городы']
column_names_2 = ['странах', 'штатах', 'регионах', 'городах']


for i, col in enumerate(columns):
  for j, dataframe in enumerate(dataframes):
    if j == 0:
        list_coun = dataframe[col].value_counts().index.to_list()[:25]
        index_coun = dataframe.query('{0} in @list_coun'.format(col)).index.to_list()


        sns.countplot(data=dataframe, x=dataframe.loc[index_coun,col], \
                      order=dataframe.loc[index_coun,col].value_counts()[:25].index, hue=dataframe['status'], \
                      hue_order=['operating', 'closed'], stat='percent', ax=axes[2*i+j]);
    else:
        list_coun = dataframe[col].value_counts().index.to_list()[:25]
        index_coun = dataframe.query('{0} in @list_coun'.format(col)).index.to_list()


        sns.countplot(data=dataframe, x=dataframe.loc[index_coun,col], \
                      order=dataframe.loc[index_coun,col].value_counts()[:25].index, hue=y_test['status'], \
                      hue_order=['operating', 'closed'], stat='percent', ax=axes[2*i+j]);

    axes[2*i+j].set_title(f'Распределение стартапо в {column_names[i]} {sample_type[j%2]}')
    axes[2*i+j].tick_params(axis='x', labelrotation=90)
    axes[2*i+j].set_xlabel(f'{column_names[i]}');
    axes[2*i+j].set_ylabel('Частота');
    axes[2*i+j].grid();


plt.tight_layout()


plt.show()

Здесь мы также видим, что Америка является наиболее представленной страной, Калифорния - наиболее представленным штатом, Район залива Сан-Франциско - наиболее представленным регионом и  Misc. - наиболее представленным городом,  ну должно было бы Сан-Франциско.Потомучто понятно сейчас что большая часть стартапов из наших наборов данных - из Кремниевой долины. Здесь кокда мы групмровали на групы Misc. мы исказили данные. Например, для признак 'regions', 'Misc.' второй.

Однако мы замечаем, что чем больше географическая зона, тем меньше группа 'Misc.' это означает, что страны — наш более надежный признак для локализации.
Меньшие зоны, более дата изскаженое в основном регионы и города. Hам пришлось сгруппировать реже встречающиеся регионы и города вместе, чтобы избежать переобучения нашей модели из-за слишком большого количества одиночных значений, ну видно что мы сейчас не можем их исползовать для МО.

Однако это не проблемы, потому что все эти признаки повторяют тот же информации для местоположение. Тогда лучее взять страну так как у него меншее пропусков чем других признаков для местоположении.

## Распеделенние по временни

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(15, 15))

axes = axes.flatten()

dataframes = [X_train, X_test]

sample_type = [' в тренировчне выборке', 'в тестовое выборке']
columns = ['first_funding_year', 'last_funding_year', 'first_funding_month', 'last_funding_month']
column_names = ['финансирование первого года', 'финансирование последного года', \
                'финансирование первого месяца', 'финансирование последного месяца']
in_out = ['начинают свой финансовый раунд', 'завершают свой финансовый раунд']

for i, col in enumerate(columns):
  for j, dataframe in enumerate(dataframes):
    if j == 0:

        sns.countplot(data=dataframe, x=dataframe[col], \
                      order=range(1,13) if i>1 else dataframe[col].value_counts().index, hue=dataframe['status'], \
                      hue_order=['operating', 'closed'], \
                      stat='percent', ax=axes[2*i+j]);
    else:

        sns.countplot(data=dataframe, x=dataframe[col], \
                      order=range(1,13) if i>1 else dataframe[col].value_counts().index, hue=y_test['status'], \
                      hue_order=['operating', 'closed'], \
                      stat='percent', ax=axes[2*i+j]);

    axes[2*i+j].set_title(f'Распределение частота стартапов которие {in_out[j%2]} {sample_type[j%2]}')
    axes[2*i+j].tick_params(axis='x', labelrotation=90)
    axes[2*i+j].set_xlabel(f'{column_names[i]}');
    axes[2*i+j].set_ylabel('Частота');
    axes[2*i+j].grid();


plt.tight_layout()


plt.show()

Мы говорили что болинство стартапов не живуть болшее 5 лет, видно здесь что эьо сложно осудить так у многих начиналось финансированое недавно по сравню дата выпускы датасета. Мы можем судить только 43% данных по этой гипотезе.

In [ ]:
len(X_train[X_train['first_funding_year']<2012])/len(X_train)

57% наших данных — это компании, у которых все еще могут быть закрыты шансы в соответствии с упомянутым нами состоянием.

In [ ]:
len(X_test[X_test['first_funding_year']<2012])/len(X_test)

## Вывод

Похоже, что большая часть нашего распределения не является нормальным распределением, а распределением Пуассона, поскольку здесь задействовано время.

Мы отметили во время исследовательского аналз данных много выбросов, много компаний, которые зарабатывают много денег, и компаний, которые имеют очень долгую жизнь, и они не обязательно одинаковы. Компании у которых много денег, как правило, затмевают молодые стартапы и стартапы с гораздо меньшими деньгами. Однако мы не можем от этих выбросов избавиться, эти стартапы не являются аномалией.

Следующее, мы замечали что в наборе тренировоных данных, признак "lifetime" оказалось большое влияние на целевую переменную, чем total_funding_year.

Мы оставили это в конце, но самое важное, что мы заметили, это то, что мы видели ранее при EDA: дисбаланс нашей целевой переменной в тренировочном данных и чрезвычайно хорошо сбалансированной классов в целевой переменной в тестовом датасета.

Дисбаланс 90%-10% в наборе тренировочных данных может стать проблемой, которую можно решить с помощью некоторых методов, таких как SMOTE или добавление опции class_weight в наши модели, чтобы модель не слишком отдавала предпочтение к крупному классу "operating".
Хорошо сбалансированные классы в нашем тестовом наборе данных также являются проблемой, поскольку нашей модели тоже не за что будет 'захватывать'. Например, мы видели среди выбросов что баланс классов 50-50 соханилось. В ходе изучения распределения различных признаков мы заметили, что большую часть времени, это хорошый баланс классов тоже соханилось. Этот слишком хороший баланс будет проблемой для модели, поскольку у нее нет никакого способа различить разницу между классами.

Проблема не в том, что в нашем тестовом наборе данных было,баланс классов 50-50, а в том, что эти классы ведут себя одинаково в зависимости от разных признаков.

# Корреляционный анализ

In [ ]:
# Let's prepare our dataset for the modelisation
X_train_corr = X_train.drop(['name', 'category_list','country_code', 'state_code', 'region', 'city', \
                          'founded_at','first_funding_at','last_funding_at', 'closed_at'], axis=1)

In [ ]:
# Let's prepare our dataset for the modelisation
X_test_corr = X_test.drop(['name', 'category_list','country_code', 'state_code', 'region', 'city', \
                        'first_funding_at','last_funding_at'], axis=1)
# Let's add the target to our dataframe
X_test_corr['status'] = y_test['status']

In [ ]:
interval_cols = ['funding_total_usd', 'lifetime',  'first_funding_year', 'last_funding_year', 'pre_seed_last_round', 'diff']

# Вычисление матрицы корреляции с использованием phik
corr_matrix = X_train_corr.phik_matrix(interval_cols=interval_cols)

# Визуализация матрицы
plt.figure(figsize=(15, 8))
sns.heatmap(corr_matrix, annot=True, cmap='crest');
plt.title('Phi_K Correlation Matrix')
plt.show()

Мы укрепили признак 'diff' зная что мы его не можем исползовать, ну просто видеть как он являеться на целевое признак.

Если мы посмотрим на матрицу, то увидим, что 'lifetime' оказывает наибольшее влияние на целевую переменную. Только fund_total_usd вообще не оказывает никакого влияния на целевую переменную. Однако 'subcategories', 'first_funding_month' и 'last_funding_round_month' имеют очень низкое влияние.

In [ ]:
interval_cols = ['funding_total_usd', 'lifetime',  'first_funding_year', 'last_funding_year', 'pre_seed_last_round']

# Вычисление матрицы корреляции с использованием phik
corr_matrix = X_test_corr.phik_matrix(interval_cols=interval_cols)

# Визуализация матрицы
plt.figure(figsize=(15, 8))
sns.heatmap(corr_matrix, annot=True, cmap='crest');
plt.title('Phi_K Correlation Matrix')
plt.show()

В нашем тренировочной данных нет действительно влиятельных переменных, что является проблемой, поскольку мы можем не получить хорошый результат, когда запустим нашу модель.

## Вывод

Так как все признаки для местоположении коллинеарный, мы выбрали только один - 'countries'. У этого признакы по сравню 'states', 'regions' и 'cities' меншее других, пропусков и меншее других унникальние значение. Так будет ещё меншее шансы получит искаженные данные  после  метода Imputer.

Для временни мы отобрали признак : 'first_funding_year' и 'last_funding_year'. Не выбрали 'last_funding_month' и 'last_funding_month' из за коллинеарност.

Вот наши тобранные признаки:

- lifetime
- countries
- funding_total_usd
- funding_rounds
- last funding year
- last funding month
- subcategories
- pre_seed_last_round


# Пайплайн

Мы увидели, что наша целевой признак несбалансирован, и нам следует использовать метод oversampling, например SMOTETomek (SMOTENC из-за этого наш расчет приостоновился). Однако обсуждалось, что такая техника не так эффективна, и люди склонны компенсировать дисбаланс своими данными с помощью алгоритма повышения, что мы и собираемся сделать с использованием CatBoost.

Что касается кодировки, поскольку у нас много одиночных значений, нам нужно как следует подумать, как их закодировать. Чтобы избежать размерного проклятия, у нас есть большой выбор кодировщика: frequency encoder, target encoder, hashing encoder. Поскольку мы собираемся использовать CatBoost для нашей модели машинного обучения, мы собираемся использовать кодировщик CatBoost, целевой кодировщик, который хорошо работает в нашем случае.

По поводу пропусков, мы будем исползовать SimpleImputer с стратегой 'most frequent' для категорялных значеных и для количественных значеных KNNImputer.

Для моделировании мы будем исползовать :
- LogisticRegression
- CatboostClassifier
- BalancedRandomForestClassifier



Дупликаты взяты из раундов финансирования, поэтому мы не собираемся их удалять.

In [ ]:
# Let's prepare dataset for the modelisation
X_train_m = X_train_corr.drop(['status', 'last_funding_year', 'last_funding_month', 'diff',\
                                'regions', 'cities', 'states'], axis=1)

X_test_m = X_test

In [ ]:
# Подготовим наш целевой признак к моделированию
y_train_m = X_train_corr['status']

# Констант
RANDOM_STATE=42

# создаём списки с названиями признаков
boost_columns_1 = ['countries', 'subcategories','first_funding_year', 'first_funding_month']

num_columns = ['lifetime','funding_total_usd', 'funding_rounds', 'pre_seed_last_round']

#cnt_encoder = ce.count.CountEncoder()
cbe_encoder = ce.cat_boost.CatBoostEncoder()
loo_encoder = ce.leave_one_out.LeaveOneOutEncoder()

# Кодируем нашу целевую переменную
le = LabelEncoder()
y_train_le = le.fit_transform(y_train_m)
y_test_le = le.transform(y_test['status'])

# создаём пайплайн для категорильного значения
boost_pipe_1 = Pipeline([
    ('imp_1', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('cboost', cbe_encoder)
])

num_pipe = Pipeline([
    ('imp', KNNImputer(n_neighbors=5)),
    ('scal', MinMaxScaler())#,
    #('poly', PolynomialFeatures(2, include_bias=False))
])

# создаём общий пайплайн для подготовки данных
data_preprocessor = ColumnTransformer(
    [('boost_1', boost_pipe_1, boost_columns_1),
     ('num', num_pipe, num_columns)
    ],
    remainder='passthrough'
)

In [ ]:
smote_tomek = SMOTETomek(sampling_strategy='auto')


# создаём итоговый пайплайн: подготовка данных и модель
pipe = Pipeline([
    ('preprocessor', data_preprocessor),
    ('smote', smote_tomek),
    ('models', [LogisticRegression(random_state=RANDOM_STATE, solver='liblinear',)])
])

param_grid = [
    # словарь для модели LogisticRegression()
      {
      'models': [LogisticRegression(random_state=RANDOM_STATE,solver='liblinear',)],
      'models__C': [0.01, 0.1, 1, 10, 100],
      'models__penalty': ['l1', 'l2'],
      'models__class_weight':['balanced'],
      'preprocessor__boost_1': [cbe_encoder, loo_encoder],
      'preprocessor__num__scal': [StandardScaler(), MinMaxScaler(), RobustScaler()],
      'smote':[SMOTETomek(sampling_strategy='all')]
    },

   # dict for CatBoostClassifier()
    {'models':[CatBoostClassifier(loss_function='Logloss', auto_class_weights='Balanced', \
                                   random_state=RANDOM_STATE)],
    'models__iterations': [100, 200],
    'models__learning_rate': [0.01, 0.1],
    'models__depth': [3, 6, 9],
    'preprocessor__boost_1': [cbe_encoder, loo_encoder],
    'preprocessor__num__scal': [StandardScaler(), MinMaxScaler(), RobustScaler()],
    'smote':[SMOTETomek(sampling_strategy='auto')]
     },
    # dict for BalancedRandomForestClassifier()
      {'models':[BalancedRandomForestClassifier(replacement=True, sampling_strategy='all',
                                                 random_state=RANDOM_STATE)],
      'models__n_estimators': [25, 33, 41, 48, 56, 64],
      'models__max_features': range(2,14),
      'models__min_samples_split':range(2,14),
      'models__min_samples_leaf':range(1,14),
      'models__max_depth': range(2,14),
      'models__bootstrap': [True, False],
      'models__class_weight': ['balanced', 'balanced_subsample'],
      'preprocessor__boost_1': [loo_encoder, cbe_encoder],
      'preprocessor__num__scal': [StandardScaler(), MinMaxScaler(), RobustScaler()] ,
      'smote':[SMOTETomek(sampling_strategy='all')]
       }

]
randomized_search = RandomizedSearchCV(
    pipe,
    param_grid,
    cv=5,
    scoring='f1',
    random_state=RANDOM_STATE,
    error_score="raise",
    n_jobs=-1
)

randomized_search.fit(X_train_m, y_train_le)

In [ ]:
# Итог гиперпараметр тюнинг
report_randomised_search = pd.DataFrame(randomized_search.cv_results_)
pd.set_option('display.max_colwidth', None)
report_randomised_search.sort_values('rank_test_score', ascending=True)[['param_preprocessor__num__scal', \
                                                                          'param_preprocessor__boost_1', \
                                                                          'param_models', 'params', \
                                                                          'mean_test_score', 'std_test_score', \
                                                                          'rank_test_score']].head(3)

In [ ]:
model = randomized_search.best_estimator_
model

# Анализ важности признаков

## С помощю SelecKBest

In [ ]:
# Let's prepare our training dataset with the data_preprocessort pipe
X_train_pipe = data_preprocessor.fit_transform(X_train_m, y_train_le)
#X_train_pipe_column_names = data_preprocessor.get_feature_names_out()
X_train_pipe_column_names = boost_columns_1 + num_columns
X_train_pipe = pd.DataFrame(X_train_pipe, columns = X_train_pipe_column_names)

In [ ]:
# Let's prepare our test dataset with the data_preprocessort pipe
X_test_pipe = data_preprocessor.transform(X_test_m)
#X_test_pipe_column_names = data_preprocessor.get_feature_names_out()
X_test_pipe_column_names = boost_columns_1 + num_columns
X_test_pipe = pd.DataFrame(X_test_pipe, columns = X_test_pipe_column_names)

In [ ]:
selector = SelectKBest(f_classif, k=8)

# обучаем SelectKBest
selector.fit(X_train_pipe, y_train_le)

# сформируйте выборки с лучшими признаками
features_names = X_train_pipe.columns[selector.get_support(indices=True)]
X_train_new = X_train_pipe[list(features_names)]
X_test_new = X_test_pipe[list(features_names)]
#print(features_names)

model_ = BalancedRandomForestClassifier(bootstrap=False, class_weight='balanced',
                               max_depth=11, max_features=3,
                               min_samples_leaf=12, min_samples_split=3,
                               n_estimators=41, random_state=42,
                               replacement=True, sampling_strategy='all')

model_.fit(X_train_new, y_train_le)

In [ ]:
# отложим значения коэффициентов на графике
coefficients = model_.feature_importances_
features_importance = pd.DataFrame({'Features': features_names, 'Importance': np.abs(coefficients)})
features_importance = features_importance.sort_values('Importance',ascending=True)
print(features_importance)
plot_features_importance = features_importance.plot(x='Features', \
                                                    y='Importance',\
                                                    kind='barh', figsize=(10, 6));
plt.title('Важности признаков с методом SelectKBest'),
plt.xlabel('Важность');
plt.ylabel('Признаки');

С методом SKbest feature_importance видим что у нас

1 очень важный признак:
- продолжительност жизни

1 важный признак:
- год первого финансирования

Есть 4 слабо влияющих признака, в порядке значимости:

- общее финансирование
- страны
- катагорие
- месяц первого финансирования

Есть 1 слабо влияющих признака:
- количество раудов


## С помощю SHAP

In [ ]:
explainer = shap.TreeExplainer(model_)

In [ ]:
shap_values = explainer(X_train_pipe.sample(frac=0.1))

In [ ]:
shap.plots.beeswarm(shap_values[:,:,0], max_display=8)

In [ ]:
shap.plots.beeswarm(shap_values[:,:,1], max_display=8)

Порядок важности функций практически такой же, как и в SelectKbest. На графиках SHAP для нашего класса 0 мы видим, что большинство признаки имеют тенденцию являет на класс 0 (отрицательные значения SHAP), наш класс меньшинства.

# Резултать исследованние

## Предсказание с самой лучшей моделей

In [ ]:
# Наш пердсказание
y_pred = model.predict(X_test)

In [ ]:
# Вероятность предсказания
y_pred_proba = model.predict_proba(X_test)

In [ ]:
#
y_pred_proba_one = y_pred_proba[:,1]

In [ ]:
print('Площадь ROC-кривой:', roc_auc_score(y_test_le, y_pred_proba_one))

print(f'Метрика Recall на тестовой выборке: {recall_score(y_test_le, y_pred)}')
print(f'Метрика Precision на тестовой выборке: {precision_score(y_test_le, y_pred)}')

cm = confusion_matrix(y_test_le, y_pred)
cm_plot = sns.heatmap(cm, annot=True, fmt='d', cmap='crest')
plt.ylabel('True label')
plt.xlabel('Predicted');

In [ ]:
# Наш метрик f1-score
f1_score(y_test_le, y_pred)

Истинно отрицательный = 689, стартапы, которым модель предсказывала провал, но которые на самом деле потерпели неудачу.

Истинно положительный = 5937, стартапы, которые, по прогнозам модели, не потерпят неудачу, и которые ещё работает.

Ошибка первого рода = 5801, стартапы, которые, по прогнозам модели, все еще работают, но в конце концов закрылись.

Ошибка второго рода = 698, стартапы, которые модель предсказывает закрытие, но они все еще работают.

Без сюрпризов: модель хорошо предсказывает класс большинства из тренировочной набора, но не может предсказать хорощо класс меньшинства. Модел действительно отдает предпочтение классу большинства, вероятно, из-за дисбаланса в тренировочной наборе.

Результат довольно плохой, поскольку у нас много ошибок первого порядка, которы мы хотели минимизировать. Здесь, модель не смог предсказать, что 5801 стартап закроются. Здесь виновата не только модель, но и тестовый набор данных, который, похоже, не отражает реальности.

## Dummy Classifier

Нам нужно посмотреть, дала ли наша модель лучший результат, чем  dummy classifier на тестовом наборе данных.

In [ ]:
dclf = DummyClassifier(random_state = RANDOM_STATE)
dclf.fit(X_train_pipe, y_train_le)
y_pred_dummy = dclf.predict(X_test_pipe)

In [ ]:
f1_score(y_test_le, y_pred_dummy)

Это нехорошо, dummy classifier дал лучший результат. Давайте посмотрим, что дает модулю pycaret, чтобы увидеть, как далеко мы находимся.

## Pycaret

In [ ]:
# Метод нормализации ç — z-оценка
# Метод дисбаланса ç — SMOTE
# Кодировщиком по умолчанию является target_encoder от category_encoders
# По умолчанию StratifiedKfold на 10
s1 = classification.setup(X_train_m, target=y_train_m, normalize=True, fix_imbalance=True, session_id=123)

In [ ]:
 # Сравнение всех моделей, чтобы определить, какая из них лучше
best = classification.compare_models()

In [ ]:
# показать лучшую модель
print(best)

In [ ]:
## показать пайплайн
classification.finalize_model(best)

In [ ]:
classification.evaluate_model(best)

In [ ]:
# Прогнозируем набор тестовых данных pycaret
classification.predict_model(best)

In [ ]:
# Что бы предсказать от нашего тестовой датасета
classification.predict_model(best, X_test_m)

# Обшее Вывод

Одной из основных трудностей был размер набора данных: в наших тренировочных данных было более 52 000 объектов и более 13000 объектов в тестовом датасете.

Наша целевая переменная является двоичной, класс 0 был 'closed', а 'operating' — класс 1. Мы заметили некоторые пропусков среди шестами признаками: 'total_funding_usd', 'category_list', 'country_code', 'state_code', 'region' и 'city'. Последние четыре — это значения местоположеннии, которые дают нам избыточную информацию. Когда мы начали изучать дубликаты, из-за большого количества объектов нам пришлось сгруппировать те, которые реже появлялись, чтобы ограничить возможное переобучениое нашей модели.

Мы также создали две признаки: одну, которая была в данных, но отсутствовала в тренировочном данных — 'продолжительность жизни', и другую, которую я создал, чтобы принять во внимание время до первого раунда, потому что некоторые стартапы терпели неудачу, потому что они были недостаточно подготовлены. и мы могли бы подумать, что, возможно, время сыграло роль в недостаточней подготовки.

В ходе EDA мы заметили, что влияние признаков оказали незначительное на целевую переменную, в основном для тестового набора данных. «Продолжительность жизни» была наиболее влиятельной на целевую переменную в трениворочном наборе.

Согласно статистике, 70% стартапов закрываются со второго по пятый год раунда финансирования, но мы заметили, что около 53% данных составляют стартапы моложе пяти лет.

Мы также замечали дисбаланс целевой переменной в пропорции 90%–10% в наборе тренировочных данных. Это вынуждает нас использовать какой-то специальный метод, такой как SMOTETomek, или использовать атрибут class_weight нашей модели, чтобы преодолеть это. Однако во втором наборе данных оба класса были почти идеально сбалансированы. Но проблема, которую мы увидели в EDA, заключается в том, что оба класса имели почти одинаковое поведение, что больше всего повлияло на результат нашего исследования (предсказание).

Действительно, после настройки нашей пайплайны и использования её с RandomizedSearchCV с необходимыми параметрами (SimpleImputer, KNNImputer, catboost и кодировщик Leave_one_out, SMOTETomek) и тремя разными моделями машинного обучения (LogisticRegression, BalancedRandomForest и Catboost), мы обучили нашу модель и на ее основе получили нашу лучшую модель: BalancedRandomForest(). Выбор был сделан с помощью f1_score. Мы получили очень хороший результат: 98,3% заставили нас думать, что наша модель, вероятно, переобучило.

Затем мы оценили наиболее важные функции нашей модели с помощью SKbest и SHAP и попытались увидеть, какие признаки мы могли бы удалить, чтобы повысить качество нашей модели. Однако после изучения или корреляции между признаками мы имеем правильный выбор признака.

Затем мы применили модель к набору данных и получили 64,8%, что неудивительно. Dummy Classifier дал лучший результат 67,1%, но этого и следовало ожидать, если бы оба класса вели себя одинаково. Неудивительно, чтоDummy Classifier, основанный на этой основе, каждый класс ведет себя с одинаковой вероятностью.

Затем мы используем pycaret, чтобы увидеть, каков был результат и какую модель он выбрал, чтобы увидеть, может ли он дать лучший результат. Лучшеe модель была Catboost, но она получила еще худший результат на f1 метрике, чем мы.

Наверно нам понадобится еще один тестовы набор данных. Даже добавить другие признаки могут нас не спасти, потому что в тестовых наборе данных есть особый случай закрытых стартапов, текущий тестовый набор кажется не очень отражает реальность.




